In [1]:
import os
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from gc import collect

In [2]:
def write_prediction_to_file(prediction):
    header = "id,nextvisit"
    with open("submission.csv", "w") as f:
        print(header, file=f)
        for idx, p in enumerate(prediction, 1):
            print(f"{idx}, {p}", file=f)

In [3]:
def visits2weekdays(visits):
    return np.apply_along_axis(lambda x: (x - 1) % 7 + 1, 0, visits)

def visits2history(visits):
    history = np.zeros(PERIOD_LENGTH, dtype=int)
    history[visits-1] = 1
    return history

In [4]:
PERIOD_LENGTH = 1098

with open('./train.csv', 'r') as f:
    data = pd.read_csv(f)
    data.visits = data.visits.apply(lambda x: np.fromstring(x, dtype=int, sep=" "))
    
train_data = np.array([v[:-1] for v in data.visits])
test_data = np.array([v[-1] for v in data.visits])
collect()

0

In [5]:
train_df = pd.DataFrame(columns=["visits", "weekdays", "history"])
train_df.visits = train_data
train_df.weekdays = train_df.visits.apply(lambda v: visits2weekdays(v))
train_df.history = train_df.visits.apply(lambda v: visits2history(v))
collect()

21

In [ ]:
train_df["mode"] = train_df.weekdays.mode

In [ ]:
stats.mode(train_df.weekdays[0])

In [ ]:
DAYS_IN_WEEK = 7
WEEK_IN_MONTH = 4
DAYS_IN_MONTH = DAYS_IN_WEEK * WEEK_IN_MOTH
MONTH_IN_YEAR = 12